# Machine interpreter

## Instructions

After a good day sightseeing, it's back to the hotel and time for some refreshment. Unfortunately, the minibar in your room refuses to open. It's been hacked, with some ransomware! You'll need to provide the correct unlock code so you can get a nice, cold drink.

You could pay a large chunk of bitcoin, or you could defeat the ransomware some other way. 

You quickly find the schematics of the minibar lock. It's a fairly simple machine. It has four registers, `a`, `b`, `c`, `d`, and a special purpose `pc` register for the program counter. Each register can hold a 64-bit value (far bigger than any number in the programs you'll be running). You can assume all registers hold zero when the program starts.

On each clock tick, the machine executes the instruction pointed to by the `pc`, then increments `pc`. The machine halts when the machine tries to read from a location beyond the end of the program.

The machine has only a few instructions. They're listed by handy mnemonics:

| Instruction | Description |
|:------------|:------------|
| `inc r`     | increment contents of register `r` |
| `dec r`     | decrement contents of register `r` |
| `set r i`   | set contents of register `r` to literal value `i` |
| `cpy r s`   | copy contents of register `r` into register `s` | 
| `jmp i`     | jump to instruction `i` places forward |
| `jpz r i`   | jump to instruction `i` places forward if<br>register `r` contains zero, otherwise continue to next instruction |

The `jmp` and `jpz` instructions jump relative to the current instruction, overriding the normal change in `pc`. `jmp -1` would jump back to the previous instruction; `jpz a 2` would skip the next instruction if register `a` contains zero.

Before you start execution of a program, you can set the values of some registers.

For example, this program multiplies the values in the a and b registers, leaving the result in the c register:

```
set c 0
cpy a d
jpz b 8
dec b
cpy d a
jpz a 4
inc c
dec a
jmp -3
jmp -7
set d 0
```

# Part 1

You think you've worked out how to generate the code wanted by the ransomware. The program is given in `07-program.txt`, one instruction per line. 

Starting with register `a` holding 7, and all other registers holding zero, what does register `a` contain when the program finishes?

It seems your guess of 7 as the starting value was wrong.

# Part 2

The program is still given in `07-program.txt`, one instruction per line. 

Starting with register `a` holding 937, and all other registers and memory locations holding zero, what does register `a` contain when the program finishes?

In [1]:
def new_machine():
    return {'pc': 0, 
            'a': 0,
            'b': 0, 
            'c': 0,
            'd': 0,
            'instructions': []}

In [2]:
def show_machine(machine):
    return ', '.join('{}: {}'.format(sk, machine[int(sk) if sk.isnumeric() else sk]) 
                     for sk in sorted(str(k) for k in machine)
                     if sk != 'instructions')

In [3]:
def inc(reg, machine):
    machine[reg] += 1
    machine['pc'] += 1

In [4]:
def dec(reg, machine):
    machine[reg] -= 1
    machine['pc'] += 1

In [5]:
def jmp(addr, machine):
    machine['pc'] += addr

In [6]:
def jpz(reg, addr, machine):
    if machine[reg] == 0:
        machine['pc'] += addr
    else:
        machine['pc'] += 1

In [7]:
def set_literal(reg, literal, machine):
    machine[reg] = literal
    machine['pc'] += 1

In [8]:
def cpy(from_reg, to_reg, machine):
    machine[to_reg] = machine[from_reg]
    machine['pc'] += 1

In [9]:
instruction_table = {'inc': inc, 'dec': dec, 'jmp': jmp,
                    'jpz': jpz, 'set': set_literal, 'cpy': cpy}
numeric_args_table = {'jmp': [0], 'jpz': [1], 'set': [1], 'sto': [1], 'ld': [1]}

In [10]:
def parse(instruction):
    words = instruction.split()
    instr = words[0]
    args = words[1:]
    if instr in numeric_args_table:
        for p in numeric_args_table[instr]:
            args[p] = int(args[p])
    return instruction_table[instr], args

In [11]:
m = new_machine()
inc('a', m)
cargs = ['a', 'b']
cpy(*cargs, m)
inc('a', m)
m

{'a': 2, 'b': 1, 'c': 0, 'd': 0, 'instructions': [], 'pc': 3}

In [12]:
def program_from_instructions(prog, machine):
    machine['instructions'] = [parse(instr) for instr in prog]

In [13]:
def unlabel_listing(listing):
    labelled_instructions = [i.strip() for i in listing.split('\n') 
                             if i.strip() 
                             if not i.strip().startswith('#')]
    return replace_labels(labelled_instructions)    

In [14]:
# def program_from_listing(listing, machine):
#     labelled_instructions = [i.strip() for i in listing.split('\n') 
#                              if i.strip() 
#                              if not i.strip().startswith('#')]
#     instructions = replace_labels(labelled_instructions)
#     program_from_instructions(instructions, machine)

In [15]:
def program_from_listing(listing, machine):
    instructions = unlabel_listing(listing)
    program_from_instructions(instructions, machine)

In [16]:
def replace_labels(listing):
    locations = {}
    for n, i in enumerate(listing):
        if ':' in i:
            locations[i.split(':')[0]] = n

    unlabelled_listing = []
    for n, i in enumerate(listing):
        instr = i.split()
        if ':' in i:
            instr = i.split(':')[1].split()
        else:
            instr = i.split()
        terms = []
        for term in instr:
            if term in locations:
                terms += [str(locations[term] - n)]
            else:
                terms += [term]
        transformed_instr = ' '.join(terms)
        unlabelled_listing += [transformed_instr]
        
    return unlabelled_listing    

In [17]:
'fred: inc a'.split(':')[1].split()

['inc', 'a']

In [18]:
program = """
      set a 10
      # comment line
      
loop: dec a
      inc b
      jpz a 2
      jmp loop
"""
labelled_instructions = [i.strip() for i in program.split('\n') if i.strip() if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))

set a 10
dec a
inc b
jpz a 2
jmp -3


In [19]:
print('\n'.join(unlabel_listing(program)))

set a 10
dec a
inc b
jpz a 2
jmp -3


In [20]:
def run(machine, initial_state=None, trace=False):
    if initial_state:
        machine.update(initial_state)
    while machine['pc'] < len(machine['instructions']):
        if trace:
            print(show_machine(machine))
        cmd, args = machine['instructions'][machine['pc']]
        cmd(*args, machine)

In [21]:
def execute(listing, initial_state=None, trace=False):
    m = new_machine()
    program_from_listing(listing, m)
    run(m, initial_state=initial_state, trace=trace)
    return m

In [22]:
program = """
inc a
inc a
cpy a b
inc a
"""
execute(program)

{'a': 3,
 'b': 2,
 'c': 0,
 'd': 0,
 'instructions': [(<function __main__.inc>, ['a']),
  (<function __main__.inc>, ['a']),
  (<function __main__.cpy>, ['a', 'b']),
  (<function __main__.inc>, ['a'])],
 'pc': 4}

In [23]:
program = """
set a 10
dec a
inc b
jpz a 2
jmp -3
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 20})

{'a': 0,
 'b': 10,
 'c': 20,
 'd': 0,
 'instructions': [(<function __main__.set_literal>, ['a', 10]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-3])],
 'pc': 5}

In [24]:
program = """
      set a 10
loop: dec a
      inc b
      jpz a 2
      jmp loop
"""
execute(program, initial_state={'c': 20})

{'a': 0,
 'b': 10,
 'c': 20,
 'd': 0,
 'instructions': [(<function __main__.set_literal>, ['a', 10]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-3])],
 'pc': 5}

In [25]:
program = """
cpy c a
set b 0
dec a
jpz b 3
dec b
jmp 2
inc b
jpz a 3
jmp -6
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 5})

{'a': 0,
 'b': 1,
 'c': 5,
 'd': 0,
 'instructions': [(<function __main__.cpy>, ['c', 'a']),
  (<function __main__.set_literal>, ['b', 0]),
  (<function __main__.dec>, ['a']),
  (<function __main__.jpz>, ['b', 3]),
  (<function __main__.dec>, ['b']),
  (<function __main__.jmp>, [2]),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 3]),
  (<function __main__.jmp>, [-6])],
 'pc': 10}

In [26]:
# b holds parity of number in c: (c % 2)
program = """
      cpy c a
      set b 0
loop: dec a
      jpz b odd
      dec b
      jmp end
odd:  inc b
end:  jpz a 2
      jmp loop
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 5})

{'a': 0,
 'b': 1,
 'c': 5,
 'd': 0,
 'instructions': [(<function __main__.cpy>, ['c', 'a']),
  (<function __main__.set_literal>, ['b', 0]),
  (<function __main__.dec>, ['a']),
  (<function __main__.jpz>, ['b', 3]),
  (<function __main__.dec>, ['b']),
  (<function __main__.jmp>, [2]),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-6])],
 'pc': 9}

In [27]:
print('\n'.join(unlabel_listing(program)))

cpy c a
set b 0
dec a
jpz b 3
dec b
jmp 2
inc b
jpz a 2
jmp -6


In [28]:
# c holds floor(a/2)
program = """
      set c 0
      set b 0
loop: dec a
      jpz b odd
      dec b
      inc c
      jmp end
odd:  inc b
end:  jpz a 2
      jmp loop
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 17}))

'a: 0, b: 1, c: 8, d: 0, pc: 10'

In [29]:
print('\n'.join(unlabel_listing(program)))

set c 0
set b 0
dec a
jpz b 4
dec b
inc c
jmp 2
inc b
jpz a 2
jmp -7


In [30]:
# c holds a * 3
program = """
      set c 0
      cpy a b
      # start of main loop
loop: jpz b end
      dec b
      inc c
      inc c
      inc c
      jmp loop
      
      # end of program 
      
end:  jmp 1
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 4}))

'a: 4, b: 0, c: 12, d: 0, pc: 9'

In [31]:
# c holds a * b
program = """
      set c 0
      cpy a d
loop: jpz b end      
      dec b
      cpy d a
smul: jpz a emul
      inc c
      dec a
      jmp smul
emul: jmp loop 
      
end:  set d 0
"""
m = new_machine()
program_from_listing(program, m)
run(m)
show_machine(execute(program, initial_state={'a': 9, 'b': 3}))

'a: 0, b: 0, c: 27, d: 0, pc: 11'

In [32]:
print('\n'.join(unlabel_listing(program)))

set c 0
cpy a d
jpz b 8
dec b
cpy d a
jpz a 4
inc c
dec a
jmp -3
jmp -7
set d 0


In [33]:
labelled_instructions = [i.strip() for i in program.split('\n') if i.strip() if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))

set c 0
cpy a d
jpz b 8
dec b
cpy d a
jpz a 4
inc c
dec a
jmp -3
jmp -7
set d 0


In [34]:
# c holds a * b
program = """
      set a 2
      set b 5
      set c 0
      cpy a d
loop: jpz b end      
      dec b
      cpy d a
smul: jpz a emul
      inc c
      dec a
      jmp smul
emul: jmp loop 
      
end:  cpy d a
"""
m = new_machine()
program_from_listing(program, m)
run(m)
show_machine(execute(program, initial_state={'a': 9, 'b': 3}))

'a: 2, b: 0, c: 10, d: 2, pc: 13'

In [35]:
labelled_instructions = [i.strip() for i in program.split('\n') 
                             if i.strip() 
                             if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))

set a 2
set b 5
set c 0
cpy a d
jpz b 8
dec b
cpy d a
jpz a 4
inc c
dec a
jmp -3
jmp -7
cpy d a


In [36]:
# Collatz. a initially holds value, but location 1 is used to store the current value as we're going along.
# Location 2 holds number of steps taken
# Location 3 holds max value reached
program = """
      cpy a d
      
      set b 0
      
      # if a is one, finish, 
main: dec a
      jpz a end
      inc a
      
      # find parity of a
      cpy a c
      set b 0
prty: dec c
      jpz b odd
      dec b
      jmp prte
odd:  inc b
prte: jpz c 2
      jmp prty
      
      # b == 0 means a even; b == 1 means a odd
      jpz b isev

      # c holds a * 3 + 1
      cpy a b
mul:  jpz b emul
      dec b
      inc c
      inc c
      inc c
      jmp mul
emul: inc c
      cpy c a
      jmp fin
      
      
isev: set c 0
      set b 0
hlvl: dec a
      jpz b oddh
      dec b
      inc c
      jmp endh
oddh: inc b
endh: jpz a 2
      jmp hlvl
      cpy c a

fin:  cpy a b
      cpy d c
maxc: jpz c this
      jpz b othr
      dec b
      dec c
      jmp maxc
this: cpy a d
othr: jmp main
      
      # end of program 
      
end:  cpy d a
      set c 0
      set d 0
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 7}))

'a: 52, b: 0, c: 0, d: 0, pc: 48'

In [37]:
13*3+1

40

In [38]:
def max_collatz(start):
    mc = start
    i = start
    while i != 1:
        if i % 2 == 0:
            i = i // 2
        else:
            i = 3 * i + 1
        if i > mc:
            mc = i
    return mc

In [39]:
max_collatz(7)

52

In [40]:
max([(max_collatz(i), i) for i in range(1, 1000)])

(250504, 937)

In [41]:
ans = []
for i in range(1, 100):
    m = execute(program, initial_state={'a': i})
    c = max_collatz(i)
    if m['a'] != c:
        ans += [(i, m['a'], c)]
ans

[]

In [42]:
show_machine(execute(program, initial_state={'a': 937}))

'a: 250504, b: 0, c: 0, d: 0, pc: 48'

In [43]:
# labelled_instructions = [i.strip() for i in program.split('\n') 
#                              if i.strip() 
#                              if not i.strip().startswith('#')]
# instructions = replace_labels(labelled_instructions)
# print('\n'.join(instructions))
# open('07-program.txt', 'w').write('\n'.join(instructions))

In [44]:
# Adds a to b
program = """
loop: dec a
      inc b
      jpz a 2
      jmp loop
"""
execute(program, initial_state={'a': 3})

{'a': 0,
 'b': 3,
 'c': 0,
 'd': 0,
 'instructions': [(<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-3])],
 'pc': 4}

In [45]:
print('\n'.join(unlabel_listing(program)))

dec a
inc b
jpz a 2
jmp -3


In [46]:
# Adds a to b
program = """
loop: dec a
      inc b
      jpz a 2
      jmp loop
"""
execute(program, initial_state={'a': 3, 'b': 4})

{'a': 0,
 'b': 7,
 'c': 0,
 'd': 0,
 'instructions': [(<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-3])],
 'pc': 4}

In [47]:
# Puts double a in b, leaves a unchanged
program = """
      set b 0
      set c 0
      jpz a end
loop: dec a
      inc b
      inc b
      inc c
      jpz a 2
      jmp loop
lp2:  dec c
      inc a
      jpz c 2
      jmp lp2
end:  set c 0      
"""
execute(program, initial_state={'a': 4})

{'a': 4,
 'b': 8,
 'c': 0,
 'd': 0,
 'instructions': [(<function __main__.set_literal>, ['b', 0]),
  (<function __main__.set_literal>, ['c', 0]),
  (<function __main__.jpz>, ['a', 11]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.inc>, ['b']),
  (<function __main__.inc>, ['c']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-5]),
  (<function __main__.dec>, ['c']),
  (<function __main__.inc>, ['a']),
  (<function __main__.jpz>, ['c', 2]),
  (<function __main__.jmp>, [-3]),
  (<function __main__.set_literal>, ['c', 0])],
 'pc': 14}

In [48]:
print('\n'.join(unlabel_listing(program)))

set b 0
set c 0
jpz a 11
dec a
inc b
inc b
inc c
jpz a 2
jmp -5
dec c
inc a
jpz c 2
jmp -3
set c 0
